# NEW CNN Model

In [14]:
# Setup

import os
from keras.preprocessing.image import ImageDataGenerator

# data directory
os.chdir("C:/Users/Sudhanshu Biyani/Desktop/folder")

image_dimensions = 80

# batch size
training_batch_size = 64 # larger = better but more computationally costly and memory intensive
validate_batch_size = 1 # optimize at runtime for parallel cores, otherwise doesn't matter much

# normalization
# normalize each chip
samplewise_center = True
samplewise_std_normalization = True
# normalize by larger batches
featurewise_center = False
featurewise_std_normalization = False

# adjacent pixel correllation reduction
# never explored
zca_whitening = False
zca_epsilon = 1e-6

# data augmentation 
# training only
transform = 0.1
zoom_range = 0.1
rotate = 360
flip = True

datagen_train = ImageDataGenerator(
        samplewise_center=samplewise_center,
        featurewise_center=featurewise_center,
        featurewise_std_normalization=featurewise_std_normalization,
        samplewise_std_normalization=samplewise_std_normalization,
        zca_whitening=zca_whitening,
        zca_epsilon=zca_epsilon,
        rotation_range=rotate,
        width_shift_range=transform,
        height_shift_range=transform,
        shear_range=transform,
        zoom_range=zoom_range,
        fill_mode='nearest',
        horizontal_flip=flip,
        vertical_flip=flip,
        rescale=1./255,
        preprocessing_function=None)

# data augmentation 
# evaluation only
transform = 0
rotate = 0
flip = False

datagen_verify = ImageDataGenerator(
        samplewise_center=samplewise_center,
        featurewise_center=featurewise_center,
        featurewise_std_normalization=featurewise_std_normalization,
        samplewise_std_normalization=samplewise_std_normalization,
        zca_whitening=zca_whitening,
        zca_epsilon=zca_epsilon,
        rotation_range=rotate,
        width_shift_range=transform,
        height_shift_range=transform,
        shear_range=transform,
        zoom_range=transform,
        fill_mode='nearest',
        horizontal_flip=flip,
        vertical_flip=flip,
        rescale=1./255,
        preprocessing_function=None)


generator_train = datagen_train.flow_from_directory(
        'train', 
        target_size=(image_dimensions,image_dimensions),
        color_mode="rgb",
        batch_size=training_batch_size,
        class_mode='categorical',
        shuffle=True)

generator_verify = datagen_verify.flow_from_directory(
        'verify', 
        target_size=(image_dimensions,image_dimensions),
        color_mode="rgb",
        batch_size=validate_batch_size,
        class_mode='categorical',
        shuffle=True)

print('Done')

Found 3850 images belonging to 8 classes.
Found 2364 images belonging to 3 classes.
Done


In [15]:
# define MobileNet architecture

from keras.applications import MobileNet

model = MobileNet(
                  input_shape=(image_dimensions, image_dimensions,3),     
                  alpha=0.25,
                  depth_multiplier=1,
                  dropout=0.5,
                  include_top=True, 
                  weights=None,
                  input_tensor=None,
                  pooling=None,
                  classes=8
                  )
model.compile(loss='categorical_crossentropy',
              optimizer='adam')

#model.summary()

print('Done')

Done


In [18]:
# Train CNN

from PIL import Image
from keras.callbacks import ModelCheckpoint

nEpochs = 500

checkpointer = ModelCheckpoint(
        filepath='sat_mobilenet_v0.h5', 
        monitor='val_loss', 
        verbose=1, 
        save_best_only=True,
        mode='auto',
        save_weights_only=False)

nFiles = sum([len(files) for r, d, files in os.walk("C:/Users/Sudhanshu Biyani/Desktop/folder/train")])
nBatches = nFiles//training_batch_size

nValFiles = sum([len(files) for r, d, files in os.walk("C:/Users/Sudhanshu Biyani/Desktop/folder/verify")])
nValbatches = nValFiles//validate_batch_size

hist = model.fit_generator(
        generator_train,
        steps_per_epoch=nBatches,
        epochs=nEpochs,
        verbose=2,
        validation_data=generator_verify,
        validation_steps=nValbatches,
        max_queue_size=10,
        callbacks=[checkpointer])

print('Done')

Epoch 1/500


ValueError: Error when checking target: expected reshape_2 to have shape (8,) but got array with shape (3,)

# OTHER EVAL TOOLS

In [15]:
# LOAD Pretrained MOBILENET MODEL

from keras.applications import mobilenet
from keras.models import load_model

model = load_model('sat_mobilenet_v0.h5', custom_objects={
                   'relu6': mobilenet.relu6,
                   'DepthwiseConv2D': mobilenet.DepthwiseConv2D})
print('Done')

Done


In [18]:
# EVALUATE

nValFiles = sum([len(files) for r, d, files in os.walk("C:/Users/Sudhanshu Biyani/Desktop/folder/verify")])
nValbatches = nValFiles//validate_batch_size

evaluation = model.evaluate_generator(
        generator_verify, 
        steps=nValbatches, 
        max_queue_size=10)

print(model.metrics_names)
print(evaluation)


['loss']
1.1489999845828474


In [19]:
# PREDICT

generator_predict = datagen_verify.flow_from_directory(
        'verify', 
        target_size=(image_dimensions,image_dimensions),
        color_mode="rgb",
        batch_size=validate_batch_size,
        class_mode='categorical',
        shuffle=False)

nValFiles = sum([len(files) for r, d, files in os.walk("C:/Users/Sudhanshu Biyani/Desktop/folder/verify")])
nValbatches = nValFiles//validate_batch_size

predictions = model.predict_generator(
        generator_predict, 
        steps=nValbatches, 
        max_queue_size=10,
        verbose=1)


Found 1176 images belonging to 8 classes.
1176/1176 [==============================] - 14s 12ms/step


In [1]:
#RUN ON IMAGE FROM DRONE
import imageio
print ("hello)")



hello)


In [2]:
import os

In [3]:
print (os.path)

<module 'ntpath' from 'C:\\Anaconda3\\lib\\ntpath.py'>


In [5]:
temp = imageio.imread('C:\Users\Sudhanshu Biyani\OneDrive - Arizona State University\Semester 2\CSE 591 - Perception in Robotics\Project\100x100 Slices\Camelback\x0y0.png')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-5-2f46fd82dfa9>, line 1)

print ("hello")

In [6]:
temp = imageio.imread('C:/Users/Sudhanshu Biyani/Desktop/x0y0.pmg')

OSError: No such file: 'C:\Users\Sudhanshu Biyani\Desktop\x0y0.pmg'

In [7]:
temp = imageio.imread('C:/Users/Sudhanshu Biyani/Desktop/x0y0.png')

In [8]:
print (temp)

[[[199 195 163]
  [196 190 163]
  [220 229 229]
  ...
  [175 160 146]
  [172 158 143]
  [180 168 151]]

 [[199 192 165]
  [192 192 162]
  [219 227 228]
  ...
  [172 167 144]
  [172 158 138]
  [175 156 142]]

 [[199 195 165]
  [191 182 155]
  [220 226 228]
  ...
  [174 166 147]
  [166 158 139]
  [173 169 145]]

 ...

 [[158 159 147]
  [131 138 138]
  [131 142 135]
  ...
  [156 158 151]
  [151 150 148]
  [152 159 152]]

 [[153 154 153]
  [110 110  93]
  [152 154 146]
  ...
  [157 158 153]
  [162 164 155]
  [153 162 155]]

 [[109 119 108]
  [120 126 116]
  [139 145 135]
  ...
  [132 139 129]
  [113 115 107]
  [107 108 106]]]


In [9]:
import pandas as pd

In [10]:
import scipy.misc

In [11]:
scipy.misc.imsave('outfile.jpg', temp)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  """Entry point for launching an IPython kernel.
